In [1]:
import topology as tg
import importlib
import uuid
importlib.reload(tg)
n1 = tg.Node(str(uuid.uuid4()), "host", 1000, int(1e9), 0, 0)
n2 = tg.Node(str(uuid.uuid4()), "host", 1000, int(1e9), 0, 0)
t = tg.Topology()
t.add_node(n1)
t.add_node(n2)
t.connect(n1, n2, uuid.uuid4(), int(1e7), 10)

In [30]:
import vivaldi.compute as vivaldi_compute
import vivaldi.coordinate as vivaldi_coordinate
importlib.reload(vivaldi_compute)
importlib.reload(vivaldi_coordinate)
lat_matrix = vivaldi_compute.get_latency_matrix(t)
print(lat_matrix)
coordinate_2d_class = vivaldi_coordinate.create_coordinate_class(2)
node_list = [n1, n2]
matrix_error = vivaldi_compute.matrix_error(lat_matrix, {n.uuid: coordinate_2d_class(0,0) for n in node_list})
print(matrix_error)

{'4d05276e-2946-474b-bce0-bdb83ce682c3': {'4d05276e-2946-474b-bce0-bdb83ce682c3': 0, '77acd811-aaa5-48e6-94bf-981e0e59eb4d': 10}, '77acd811-aaa5-48e6-94bf-981e0e59eb4d': {'4d05276e-2946-474b-bce0-bdb83ce682c3': 10, '77acd811-aaa5-48e6-94bf-981e0e59eb4d': 0}}
100.0


In [32]:
updated = vivaldi_compute.vivaldi_compute(t, {n.uuid: coordinate_2d_class(0,0) for n in node_list}, 1, 20)
for k, v in updated.items():
    print(k, v)

2021-08-01 22:18:10 Vancior-Redmi vivaldi.compute[30708] DEBUG initial error: 100.0
2021-08-01 22:18:10 Vancior-Redmi vivaldi.compute[30708] DEBUG iteration #0 error: 65.61
2021-08-01 22:18:10 Vancior-Redmi vivaldi.compute[30708] DEBUG iteration #1 error: 43.046721
2021-08-01 22:18:10 Vancior-Redmi vivaldi.compute[30708] DEBUG iteration #2 error: 28.242953648099995
2021-08-01 22:18:10 Vancior-Redmi vivaldi.compute[30708] DEBUG iteration #3 error: 18.530201888518413
2021-08-01 22:18:10 Vancior-Redmi vivaldi.compute[30708] DEBUG iteration #4 error: 12.157665459056926
2021-08-01 22:18:10 Vancior-Redmi vivaldi.compute[30708] DEBUG iteration #5 error: 7.976644307687252
2021-08-01 22:18:10 Vancior-Redmi vivaldi.compute[30708] DEBUG iteration #6 error: 5.233476330273604
2021-08-01 22:18:10 Vancior-Redmi vivaldi.compute[30708] DEBUG iteration #7 error: 3.433683820292508
2021-08-01 22:18:10 Vancior-Redmi vivaldi.compute[30708] DEBUG iteration #8 error: 2.252839954493914
2021-08-01 22:18:10 Vanc

4d05276e-2946-474b-bce0-bdb83ce682c3 (4.70117855310172,-0.6423333613229387)
77acd811-aaa5-48e6-94bf-981e0e59eb4d (-4.231060697791547,0.5781000251906449)
